In [1]:
from agential.llm.llm import LLM
from dotenv import load_dotenv

load_dotenv()

import os

llm = LLM("gpt-3.5-turbo", organization=os.getenv("OPENAI_ORGANIZATION"))

from agential.cog.expel.strategies.general import ExpeLGeneralStrategy

c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\tuvin\anaconda3\envs\agential\Lib\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [2]:
import joblib

exp = joblib.load("../../../tests/assets/expel/expel_experiences_10_fake.joblib")

In [3]:
from agential.cog.expel.memory import ExpeLExperienceMemory
from agential.cog.reflexion.agent import ReflexionReActAgent


agent = ReflexionReActAgent(
    llm=llm, 
    benchmark="hotpotqa",
)

strat = ExpeLGeneralStrategy(
    llm=llm, 
    reflexion_react_agent=agent,
    experience_memory=ExpeLExperienceMemory(experiences=exp)
)

In [4]:
from agential.cog.fewshots.hotpotqa import HOTPOTQA_FEWSHOT_EXAMPLES_REACT
from agential.cog.expel.prompts import HOTPOTQA_FEWSHOT_EXAMPLES_EXPEL_REFLEXION_REACT_REFLECT, EXPEL_REFLEXION_REACT_INSTRUCTION_HOTPOTQA, EXPEL_REFLEXION_REACT_REFLECT_INSTRUCTION_HOTPOTQA


question = "What giant silverware company was started as a religious Utopian group and was for many years run by Pierrepont Noyes?"
key = "Oneida Limited"

out = strat.generate(
        question=question,
        key=key,
        examples=HOTPOTQA_FEWSHOT_EXAMPLES_REACT,
        prompt=EXPEL_REFLEXION_REACT_INSTRUCTION_HOTPOTQA,
        reflect_examples=HOTPOTQA_FEWSHOT_EXAMPLES_EXPEL_REFLEXION_REACT_REFLECT,
        reflect_prompt=EXPEL_REFLEXION_REACT_REFLECT_INSTRUCTION_HOTPOTQA,
        reflect_strategy="reflexion",
        additional_keys={},
        reflect_additional_keys={},
        use_dynamic_examples = True,
        extract_insights = True,
        patience = 3,
        k_docs = 24,
        num_fewshots = 6,
        max_fewshot_tokens = 1500,
        reranker_strategy = None,
        reset=False
    )

<PROMPT COMPARE ======================================================================>
You are an advanced reasoning agent that can critique past task trajectories of youself. You will be given two previous task trials in which you were given access to a Docstore API environment and a question to answer: one successful and one unsuccessful trial. You failed the trial either because you guessed the wrong answer with Finish[<answer>], or you used up your set number of reasoning steps.

Here are the two previous trials to compare and critique:
TRIAL TASK:
The creator of "Wallace and Gromit" also created what animation comedy that matched animated zoo animals with a soundtrack of people talking about their homes? 

SUCCESSFUL TRIAL:
Thought: I need to focus on searching for the specific animation comedy that matched animated zoo animals with a soundtrack of people talking about their homes, created by the same creator of "Wallace and Gromit."
Action: Search[animation comedy matched animat

In [5]:
out.total_tokens

24570

In [6]:
llm.total_tokens

24570